In [1]:
#Librerias
import pandas as pd
import numpy as ny
import googlemaps
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderInsufficientPrivileges
from geopy.extra.rate_limiter import RateLimiter
import logging
import pandas as pd
import time
import re
import csv
import os
import hashlib

unidad educativa 2014

In [12]:
# 1. Cargamos los datos
file_path = r'H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2014.csv'  
data = pd.read_csv(file_path, delimiter='\t')  

# 2. Realizamos una revisión general
print("Dimensiones del dataset:", data.shape)
print("\nNombres de las columnas:", data.columns.tolist())
print("\nPrimeras filas del dataset:")
print(data.head())

# 3. Mostramos la calidad de los datos
print("\nValores nulos por columna:")
print(data.isnull().sum())

print("\nTipos de datos:")
print(data.dtypes)

# 4. Realisaos una estadísticas descriptivas
print("\nEstadísticas descriptivas para columnas numéricas:")
print(data.describe())

print("\nValores únicos por columna:")
print(data.nunique())





Dimensiones del dataset: (6182, 7)

Nombres de las columnas: ['name', 'addr:street', 'addr:city', 'addr:state', 'amenity', '@lat', '@lon']

Primeras filas del dataset:
                name addr:street addr:city addr:state amenity       @lat  \
0   Allentown School         NaN       NaN        NaN  school  30.764356   
1    Alliance School         NaN       NaN        NaN  school  30.621027   
2     Allison School         NaN       NaN        NaN  school  30.339651   
3  Alta Vista School         NaN       NaN        NaN  school  27.325324   
4    Anderson School         NaN       NaN        NaN  school  30.034406   

        @lon  
0 -87.075245  
1 -85.136313  
2 -83.826548  
3 -82.518152  
4 -82.070106  

Valores nulos por columna:
name            327
addr:street    4711
addr:city      4850
addr:state     4996
amenity           0
@lat              0
@lon              0
dtype: int64

Tipos de datos:
name            object
addr:street     object
addr:city       object
addr:state      ob

In [13]:
# 1. Cargamos los datos
file_path = r'H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2014.csv'  
data = pd.read_csv(file_path, delimiter='\t')  

# Configuramos el geolocalizador con un user_agent único
geolocator = Nominatim(user_agent="mi_proyecto_geocodificacion_v1")

# Función para realizar la geocodificación inversa y buscar el nombre del lugar
def reverse_geocode(lat, lon):
    try:
        location = geolocator.reverse((lat, lon), timeout=10)
        if location:
            address = location.address
            city = location.raw.get('address', {}).get('city', None)
            name = location.raw.get('name', None) or location.raw.get('address', {}).get('attraction', None)
            return address, city, name
        else:
            return "Dirección no encontrada", None, None
    except GeocoderTimedOut:
        return "Tiempo de espera agotado", None, None
    except GeocoderInsufficientPrivileges:
        return "Bloqueado: Verificar API o User-Agent", None, None

# Limpieza inicial del archivo
input_file = "H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2014.csv"
cleaned_file = "H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 201412.csv"

with open(input_file, 'r', encoding='utf-8') as infile, open(cleaned_file, 'w', encoding='utf-8') as outfile:
    for line in infile:
        if line.strip():  # Omitir líneas vacías
            outfile.write(line)

# Cargamos el archivo limpio
try:
    data = pd.read_csv(cleaned_file, sep='\t', encoding='utf-8')
except pd.errors.ParserError as e:
    print(f"Error al cargar el archivo: {e}")
    data = pd.read_csv(cleaned_file, sep=',', encoding='utf-8', on_bad_lines='skip')

# Añadimos  la columna 'name' si no existe
if 'name' not in data.columns:
    data['name'] = None

# Actualizamos las columnas addr:street, addr:city y name
def update_address(row):
    if pd.notna(row['@lat']) and pd.notna(row['@lon']):
        address, city, name = reverse_geocode(row['@lat'], row['@lon'])
        row['addr:street'] = address
        row['addr:city'] = city
        row['name'] = name
    return row

# Aplicamos la función fila por fila con pausas para respetar los límites de solicitudes
for index, row in data.iterrows():
    data.loc[index] = update_address(row)
    time.sleep(2)

# Guardamos los resultados
output_file = "H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2014.csv"
data.to_csv(output_file, index=False)
print(data.head())




                name                                        addr:street  \
0   Allentown School  Allentown School, 6180, Central School Road, M...   
1    Alliance School  Alliance School, Alliance Road, Sink Creek, Ja...   
2     Allison School  Allison School, Lanier Grade, Jefferson County...   
3  Alta Vista School  Alta Vista School, South Euclid Avenue, Saraso...   
4    Anderson School  Anderson School, Northeast 17th Avenue, Bradfo...   

  addr:city addr:state amenity       @lat       @lon  
0      None        NaN  school  30.764356 -87.075245  
1      None        NaN  school  30.621027 -85.136313  
2      None        NaN  school  30.339651 -83.826548  
3  Sarasota        NaN  school  27.325324 -82.518152  
4      None        NaN  school  30.034406 -82.070106  


Extraemos los datos de la columna direccion

Extraemos imformacion de la columna nombre

In [ ]:
# Cargamos  los datos 
archivo = "H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2014.csv"  
data = pd.read_csv(archivo)

# Renombramos  las columnas
data.rename(columns={
    'addr:street': 'direccion',
    'amenity': 'tipo',
    '@lat': 'latitud',
    '@lon': 'longitud'
}, inplace=True)

# Dividimos la columna direccion para extraer los datos
direccion_split = data['direccion'].fillna('').str.split(',', expand=True)

# Creamos las nuebas colunas a usar
data['nombre'] = direccion_split[0].str.strip()

# Extraemos la direccion combinando todas las partes de la dirección hasta el estado
data['direccion'] = direccion_split.apply(
    lambda row: ', '.join(filter(None, [row[1].strip(), row[2].strip()])), axis=1
)

# Extraemos la ciudad siempre y cuando tenga la palabra County 
data['ciudad'] = direccion_split.apply(
    lambda row: next((x.strip() for x in row if 'County' in str(x)), 'No definido'), axis=1
)

# Extraemos el estado  "Florida"
data['estado'] = direccion_split.apply(lambda row: 'Florida', axis=1)

# Extraemos  y validamos los códigos postales 
data['codigo_postal'] = direccion_split.apply(
    lambda row: next((re.search(r'\b\d{5}\b', str(x)).group(0) for x in row if re.search(r'\b\d{5}\b', str(x))), 'No definido'), axis=1
)

# Separamos el peis "United States"
data['pais'] = direccion_split.apply(lambda row: 'United States', axis=1)

# Reordenamos las columnas 
data = data[['nombre', 'direccion', 'ciudad', 'estado', 'pais', 'codigo_postal', 'longitud', 'latitud']]

# Eliminamos  duplicados
data.drop_duplicates(inplace=True)

# Guardamos el resultado
processed_file_path = 'H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2014.csv'
data.to_csv(processed_file_path, index=False)

# Análisamos de datos vacíos y duplicados
vacios = data.isnull().sum()
duplicados = data.duplicated().sum()

print("Análisis de datos:")
print("Valores vacíos por columna:")
print(vacios)
print(f"Número de filas duplicadas: {duplicados}")
print(data.head())


Análisis de datos:
Valores vacíos por columna:
nombre           0
direccion        0
ciudad           0
estado           0
pais             0
codigo_postal    0
longitud         0
latitud          0
dtype: int64
Número de filas duplicadas: 0
              nombre                               direccion  \
0   Allentown School               6180, Central School Road   
1    Alliance School               Alliance Road, Sink Creek   
2     Allison School          Lanier Grade, Jefferson County   
3  Alta Vista School           South Euclid Avenue, Sarasota   
4    Anderson School  Northeast 17th Avenue, Bradford County   

              ciudad   estado           pais codigo_postal   longitud  \
0  Santa Rosa County  Florida  United States         32570 -87.075245   
1     Jackson County  Florida  United States         32448 -85.136313   
2   Jefferson County  Florida  United States   No definido -83.826548   
3    Sarasota County  Florida  United States         34239 -82.518152   
4    Bra

Eliminamos la columna county

In [5]:
# Eliminamos la palabra 'County' de las columnas 'direccion' y 'ciudad'
data['direccion'] = data['direccion'].str.replace(r'\bCounty\b', '', regex=True).str.strip()
data['ciudad'] = data['ciudad'].str.replace(r'\bCounty\b', '', regex=True).str.strip()
print(data[['direccion', 'ciudad']].head())

# Guardamos el resultado 
processed_file_path = 'H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2014.csv'
data.to_csv(processed_file_path, index=False)

                         direccion      ciudad
0        6180, Central School Road  Santa Rosa
1        Alliance Road, Sink Creek     Jackson
2          Lanier Grade, Jefferson   Jefferson
3    South Euclid Avenue, Sarasota    Sarasota
4  Northeast 17th Avenue, Bradford    Bradford


Creamops la columna id nomnbre y id ciudad

In [6]:
# Cargamos las rutas de los archivos
input_path = r"H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2014.csv"
output_path = r"H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2014.csv"
nombres_dim_path = "H:/Nueva carpeta/overpass-turbo.eu/nombres_unidades_educativas_dim.csv"
ciudades_dim_path = "H:/Nueva carpeta/overpass-turbo.eu/ciudades_dim.csv"

# Cargamos los catálogos
def load_catalog(path, key, value):
    catalog = {}
    if os.path.exists(path):
        with open(path, mode='r', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for row in reader:
                catalog[row[key]] = int(row[value])
    return catalog

# Guardamos los catálogos de ciudad y nombres
def save_catalog(path, catalog, key, value):
    with open(path, mode='w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=[value, key])
        writer.writeheader()
        for k, v in catalog.items():
            writer.writerow({value: v, key: k})

# Generamos IDs únicos
def get_id(value, catalog):
    if value not in catalog:
        catalog[value] = len(catalog) + 1
    return catalog[value]

# Cargamos catálogos existentes
nombres_dict = load_catalog(nombres_dim_path, "nombre", "id_nombre")
ciudades_dict = load_catalog(ciudades_dim_path, "ciudad", "id_ciudad")

# Procesamos archivo de entrada
def process_file(input_path, output_path):
    if not os.path.exists(input_path):
        raise FileNotFoundError(f"El archivo de entrada no existe: {input_path}")

    with open(input_path, mode='r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        original_fieldnames = reader.fieldnames

        # Validamos las columnas columnas requeridas
        required_cols = {"nombre", "direccion", "latitud", "longitud", "ciudad", "estado", "pais", "codigo_postal"}
        missing_cols = required_cols - set(original_fieldnames)
        if missing_cols:
            raise ValueError(f"El CSV no contiene las siguientes columnas requeridas: {', '.join(missing_cols)}")

        # Nombres de las columnas
        fieldnames = [
            "id_nombre", "nombre", "direccion", "latitud", "longitud",
            "id_ciudad", "ciudad", "estado", "pais", "codigo_postal"
        ]

        # ordenamos y creamos el archivo de salida
        with open(output_path, mode='w', newline='', encoding='utf-8') as outfile:
            writer = csv.DictWriter(outfile, fieldnames=fieldnames)
            writer.writeheader()

            for row in reader:
                id_nombre = get_id(row["nombre"], nombres_dict)
                id_ciudad = get_id(row["ciudad"], ciudades_dict)

                new_row = {
                    "id_nombre": id_nombre,
                    "nombre": row["nombre"],
                    "direccion": row["direccion"],
                    "latitud": row["latitud"],
                    "longitud": row["longitud"],
                    "id_ciudad": id_ciudad,
                    "ciudad": row["ciudad"],
                    "estado": row["estado"],
                    "pais": row["pais"],
                    "codigo_postal": row["codigo_postal"]
                }
                writer.writerow(new_row)

process_file(input_path, output_path)

# Guardamos el catálogos actualizados 
save_catalog(nombres_dim_path, nombres_dict, "nombre", "id_nombre")
save_catalog(ciudades_dim_path, ciudades_dict, "ciudad", "id_ciudad")

print("Proceso completado con éxito.")

Proceso completado con éxito.


Trasformamos el archivo a .parquet

In [7]:
# Cargamos el archivo CSV 
csv_path = r"H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2014.csv"

# Ruta del archivo Parquet de salida
parquet_path = r"H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2014.parquet"

df = pd.read_csv(csv_path)

# Guardamos el archivo Parquet
df.to_parquet(parquet_path, engine='pyarrow', index=False)

print(f"Archivo convertido exitosamente a Parquet: {parquet_path}")


Archivo convertido exitosamente a Parquet: H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2014.parquet


unidad educativa 2015

Extraemos informacion de la columna direccion

In [8]:
# Cargamos  los datos 
archivo = "H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2015.csv"  
data = pd.read_csv(archivo)

# Renombramos  las columnas
data.rename(columns={
    'addr:street': 'direccion',
    'amenity': 'tipo',
    '@lat': 'latitud',
    '@lon': 'longitud'
}, inplace=True)

# Dividimos la columna direccion para extraer los datos
direccion_split = data['direccion'].fillna('').str.split(',', expand=True)

# Creamos las nuebas colunas a usar
data['nombre'] = direccion_split[0].str.strip()

# Extraemos la direccion combinando todas las partes de la dirección hasta el estado
data['direccion'] = direccion_split.apply(
    lambda row: ', '.join(filter(None, [row[1].strip(), row[2].strip()])), axis=1
)

# Extraemos la ciudad siempre y cuando tenga la palabra County 
data['ciudad'] = direccion_split.apply(
    lambda row: next((x.strip() for x in row if 'County' in str(x)), 'No definido'), axis=1
)

# Extraemos el estado  "Florida"
data['estado'] = direccion_split.apply(lambda row: 'Florida', axis=1)

# Extraemos  y validamos los códigos postales 
data['codigo_postal'] = direccion_split.apply(
    lambda row: next((re.search(r'\b\d{5}\b', str(x)).group(0) for x in row if re.search(r'\b\d{5}\b', str(x))), 'No definido'), axis=1
)

# Separamos el peis "United States"
data['pais'] = direccion_split.apply(lambda row: 'United States', axis=1)

# Reordenamos las columnas 
data = data[['nombre', 'direccion', 'ciudad', 'estado', 'pais', 'codigo_postal', 'longitud', 'latitud']]

# Eliminamos  duplicados
data.drop_duplicates(inplace=True)

# Guardamos el resultado
processed_file_path = 'H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2015.csv'
data.to_csv(processed_file_path, index=False)

# Análisamos de datos vacíos y duplicados
vacios = data.isnull().sum()
duplicados = data.duplicated().sum()

print("Análisis de datos:")
print("Valores vacíos por columna:")
print(vacios)
print(f"Número de filas duplicadas: {duplicados}")
print(data.head())

Análisis de datos:
Valores vacíos por columna:
nombre           0
direccion        0
ciudad           0
estado           0
pais             0
codigo_postal    0
longitud         0
latitud          0
dtype: int64
Número de filas duplicadas: 0
              nombre                               direccion  \
0   Allentown School               6180, Central School Road   
1    Alliance School               Alliance Road, Sink Creek   
2     Allison School          Lanier Grade, Jefferson County   
3  Alta Vista School           South Euclid Avenue, Sarasota   
4    Anderson School  Northeast 17th Avenue, Bradford County   

              ciudad   estado           pais codigo_postal   longitud  \
0  Santa Rosa County  Florida  United States         32570 -87.075245   
1     Jackson County  Florida  United States         32448 -85.136313   
2   Jefferson County  Florida  United States   No definido -83.826548   
3    Sarasota County  Florida  United States         34239 -82.518152   
4    Bra

Eliminamos la palabra countyt

In [9]:
# Eliminamos la palabra 'County' de las columnas 'direccion' y 'ciudad'
data['direccion'] = data['direccion'].str.replace(r'\bCounty\b', '', regex=True).str.strip()
data['ciudad'] = data['ciudad'].str.replace(r'\bCounty\b', '', regex=True).str.strip()
print(data[['direccion', 'ciudad']].head())

# Guardamos el resultado 
processed_file_path = 'H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2015.csv'
data.to_csv(processed_file_path, index=False)

                         direccion      ciudad
0        6180, Central School Road  Santa Rosa
1        Alliance Road, Sink Creek     Jackson
2          Lanier Grade, Jefferson   Jefferson
3    South Euclid Avenue, Sarasota    Sarasota
4  Northeast 17th Avenue, Bradford    Bradford


Ccreamos la columna id nombre e id ciudad

In [10]:
# Cargamos las rutas de los archivos
input_path = r"H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2015.csv"
output_path = r"H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2015.csv"
nombres_dim_path = "H:/Nueva carpeta/overpass-turbo.eu/nombres_unidades_educativas_dim.csv"
ciudades_dim_path = "H:/Nueva carpeta/overpass-turbo.eu/ciudades_dim.csv"

# Cargamos los catálogos
def load_catalog(path, key, value):
    catalog = {}
    if os.path.exists(path):
        with open(path, mode='r', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for row in reader:
                catalog[row[key]] = int(row[value])
    return catalog

# Guardamos los catálogos de ciudad y nombres
def save_catalog(path, catalog, key, value):
    with open(path, mode='w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=[value, key])
        writer.writeheader()
        for k, v in catalog.items():
            writer.writerow({value: v, key: k})

# Generamos IDs únicos
def get_id(value, catalog):
    if value not in catalog:
        catalog[value] = len(catalog) + 1
    return catalog[value]

# Cargamos catálogos existentes
nombres_dict = load_catalog(nombres_dim_path, "nombre", "id_nombre")
ciudades_dict = load_catalog(ciudades_dim_path, "ciudad", "id_ciudad")

# Procesamos archivo de entrada
def process_file(input_path, output_path):
    if not os.path.exists(input_path):
        raise FileNotFoundError(f"El archivo de entrada no existe: {input_path}")

    with open(input_path, mode='r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        original_fieldnames = reader.fieldnames

        # Validamos las columnas columnas requeridas
        required_cols = {"nombre", "direccion", "latitud", "longitud", "ciudad", "estado", "pais", "codigo_postal"}
        missing_cols = required_cols - set(original_fieldnames)
        if missing_cols:
            raise ValueError(f"El CSV no contiene las siguientes columnas requeridas: {', '.join(missing_cols)}")

        # Nombres de las columnas
        fieldnames = [
            "id_nombre", "nombre", "direccion", "latitud", "longitud",
            "id_ciudad", "ciudad", "estado", "pais", "codigo_postal"
        ]

        # ordenamos y creamos el archivo de salida
        with open(output_path, mode='w', newline='', encoding='utf-8') as outfile:
            writer = csv.DictWriter(outfile, fieldnames=fieldnames)
            writer.writeheader()

            for row in reader:
                id_nombre = get_id(row["nombre"], nombres_dict)
                id_ciudad = get_id(row["ciudad"], ciudades_dict)

                new_row = {
                    "id_nombre": id_nombre,
                    "nombre": row["nombre"],
                    "direccion": row["direccion"],
                    "latitud": row["latitud"],
                    "longitud": row["longitud"],
                    "id_ciudad": id_ciudad,
                    "ciudad": row["ciudad"],
                    "estado": row["estado"],
                    "pais": row["pais"],
                    "codigo_postal": row["codigo_postal"]
                }
                writer.writerow(new_row)

process_file(input_path, output_path)

# Guardamos el catálogos actualizados 
save_catalog(nombres_dim_path, nombres_dict, "nombre", "id_nombre")
save_catalog(ciudades_dim_path, ciudades_dict, "ciudad", "id_ciudad")

print("Proceso completado con éxito.")

Proceso completado con éxito.


Pasamos el archivo a formato parquet

In [11]:
# Cargamos el archivo CSV 
csv_path = r"H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2015.csv"

# Ruta del archivo Parquet de salida
parquet_path = r"H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2015.parquet"

df = pd.read_csv(csv_path)

# Guardamos el archivo Parquet
df.to_parquet(parquet_path, engine='pyarrow', index=False)

print(f"Archivo convertido exitosamente a Parquet: {parquet_path}")

Archivo convertido exitosamente a Parquet: H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2015.parquet


unidad educativa 2016

Extraemos la informacion de columna direccion

In [13]:
# Cargamos  los datos 
archivo = "H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2016.csv"  
data = pd.read_csv(archivo)

# Renombramos  las columnas
data.rename(columns={
    'addr:street': 'direccion',
    'amenity': 'tipo',
    '@lat': 'latitud',
    '@lon': 'longitud'
}, inplace=True)

# Dividimos la columna direccion para extraer los datos
direccion_split = data['direccion'].fillna('').str.split(',', expand=True)

# Creamos las nuebas colunas a usar
data['nombre'] = direccion_split[0].str.strip()

# Extraemos la direccion combinando todas las partes de la dirección hasta el estado
data['direccion'] = direccion_split.apply(
    lambda row: ', '.join(filter(None, [row[1].strip(), row[2].strip()])), axis=1
)

# Extraemos la ciudad siempre y cuando tenga la palabra County 
data['ciudad'] = direccion_split.apply(
    lambda row: next((x.strip() for x in row if 'County' in str(x)), 'No definido'), axis=1
)

# Extraemos el estado  "Florida"
data['estado'] = direccion_split.apply(lambda row: 'Florida', axis=1)

# Extraemos  y validamos los códigos postales 
data['codigo_postal'] = direccion_split.apply(
    lambda row: next((re.search(r'\b\d{5}\b', str(x)).group(0) for x in row if re.search(r'\b\d{5}\b', str(x))), 'No definido'), axis=1
)

# Separamos el peis "United States"
data['pais'] = direccion_split.apply(lambda row: 'United States', axis=1)

# Reordenamos las columnas 
data = data[['nombre', 'direccion', 'ciudad', 'estado', 'pais', 'codigo_postal', 'longitud', 'latitud']]

# Eliminamos  duplicados
data.drop_duplicates(inplace=True)

# Guardamos el resultado
processed_file_path = 'H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2016.csv'
data.to_csv(processed_file_path, index=False)

# Análisamos de datos vacíos y duplicados
vacios = data.isnull().sum()
duplicados = data.duplicated().sum()

print("Análisis de datos:")
print("Valores vacíos por columna:")
print(vacios)
print(f"Número de filas duplicadas: {duplicados}")
print(data.head())

Análisis de datos:
Valores vacíos por columna:
nombre           0
direccion        0
ciudad           0
estado           0
pais             0
codigo_postal    0
longitud         0
latitud          0
dtype: int64
Número de filas duplicadas: 0
              nombre                               direccion  \
0   Allentown School               6180, Central School Road   
1    Alliance School               Alliance Road, Sink Creek   
2     Allison School          Lanier Grade, Jefferson County   
3  Alta Vista School           South Euclid Avenue, Sarasota   
4    Anderson School  Northeast 17th Avenue, Bradford County   

              ciudad   estado           pais codigo_postal   longitud  \
0  Santa Rosa County  Florida  United States         32570 -87.075245   
1     Jackson County  Florida  United States         32448 -85.136313   
2   Jefferson County  Florida  United States   No definido -83.826548   
3    Sarasota County  Florida  United States         34239 -82.518152   
4    Bra

Eliminamos la palabra county 

In [14]:
# Eliminamos la palabra 'County' de las columnas 'direccion' y 'ciudad'
data['direccion'] = data['direccion'].str.replace(r'\bCounty\b', '', regex=True).str.strip()
data['ciudad'] = data['ciudad'].str.replace(r'\bCounty\b', '', regex=True).str.strip()
print(data[['direccion', 'ciudad']].head())

# Guardamos el resultado 
processed_file_path = 'H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2016.csv'
data.to_csv(processed_file_path, index=False)

                         direccion      ciudad
0        6180, Central School Road  Santa Rosa
1        Alliance Road, Sink Creek     Jackson
2          Lanier Grade, Jefferson   Jefferson
3    South Euclid Avenue, Sarasota    Sarasota
4  Northeast 17th Avenue, Bradford    Bradford


 Creamos la columna id nombre e id ciudad

In [15]:
# Cargamos las rutas de los archivos
input_path = r"H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2016.csv"
output_path = r"H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2016.csv"
nombres_dim_path = "H:/Nueva carpeta/overpass-turbo.eu/nombres_unidades_educativas_dim.csv"
ciudades_dim_path = "H:/Nueva carpeta/overpass-turbo.eu/ciudades_dim.csv"

# Cargamos los catálogos
def load_catalog(path, key, value):
    catalog = {}
    if os.path.exists(path):
        with open(path, mode='r', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for row in reader:
                catalog[row[key]] = int(row[value])
    return catalog

# Guardamos los catálogos de ciudad y nombres
def save_catalog(path, catalog, key, value):
    with open(path, mode='w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=[value, key])
        writer.writeheader()
        for k, v in catalog.items():
            writer.writerow({value: v, key: k})

# Generamos IDs únicos
def get_id(value, catalog):
    if value not in catalog:
        catalog[value] = len(catalog) + 1
    return catalog[value]

# Cargamos catálogos existentes
nombres_dict = load_catalog(nombres_dim_path, "nombre", "id_nombre")
ciudades_dict = load_catalog(ciudades_dim_path, "ciudad", "id_ciudad")

# Procesamos archivo de entrada
def process_file(input_path, output_path):
    if not os.path.exists(input_path):
        raise FileNotFoundError(f"El archivo de entrada no existe: {input_path}")

    with open(input_path, mode='r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        original_fieldnames = reader.fieldnames

        # Validamos las columnas columnas requeridas
        required_cols = {"nombre", "direccion", "latitud", "longitud", "ciudad", "estado", "pais", "codigo_postal"}
        missing_cols = required_cols - set(original_fieldnames)
        if missing_cols:
            raise ValueError(f"El CSV no contiene las siguientes columnas requeridas: {', '.join(missing_cols)}")

        # Nombres de las columnas
        fieldnames = [
            "id_nombre", "nombre", "direccion", "latitud", "longitud",
            "id_ciudad", "ciudad", "estado", "pais", "codigo_postal"
        ]

        # ordenamos y creamos el archivo de salida
        with open(output_path, mode='w', newline='', encoding='utf-8') as outfile:
            writer = csv.DictWriter(outfile, fieldnames=fieldnames)
            writer.writeheader()

            for row in reader:
                id_nombre = get_id(row["nombre"], nombres_dict)
                id_ciudad = get_id(row["ciudad"], ciudades_dict)

                new_row = {
                    "id_nombre": id_nombre,
                    "nombre": row["nombre"],
                    "direccion": row["direccion"],
                    "latitud": row["latitud"],
                    "longitud": row["longitud"],
                    "id_ciudad": id_ciudad,
                    "ciudad": row["ciudad"],
                    "estado": row["estado"],
                    "pais": row["pais"],
                    "codigo_postal": row["codigo_postal"]
                }
                writer.writerow(new_row)

process_file(input_path, output_path)

# Guardamos el catálogos actualizados 
save_catalog(nombres_dim_path, nombres_dict, "nombre", "id_nombre")
save_catalog(ciudades_dim_path, ciudades_dict, "ciudad", "id_ciudad")

print("Proceso completado con éxito.")

Proceso completado con éxito.


Trasformamos el archivo a .parquet

In [26]:
# Cargamos el archivo CSV 
csv_path = r"H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2018.csv"

# Ruta del archivo Parquet de salida
parquet_path = r"H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2018.parquet"

df = pd.read_csv(csv_path)

# Guardamos el archivo Parquet
df.to_parquet(parquet_path, engine='pyarrow', index=False)

print(f"Archivo convertido exitosamente a Parquet: {parquet_path}")

Archivo convertido exitosamente a Parquet: H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2018.parquet


unidad educativa 2017

Extraemos la informacion de la columna direccion 

In [18]:
# Cargamos  los datos 
archivo = "H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2017.csv"  
data = pd.read_csv(archivo)

# Renombramos  las columnas
data.rename(columns={
    'addr:street': 'direccion',
    'amenity': 'tipo',
    '@lat': 'latitud',
    '@lon': 'longitud'
}, inplace=True)

# Dividimos la columna direccion para extraer los datos
direccion_split = data['direccion'].fillna('').str.split(',', expand=True)

# Creamos las nuebas colunas a usar
data['nombre'] = direccion_split[0].str.strip()

# Extraemos la direccion combinando todas las partes de la dirección hasta el estado
data['direccion'] = direccion_split.apply(
    lambda row: ', '.join(filter(None, [row[1].strip(), row[2].strip()])), axis=1
)

# Extraemos la ciudad siempre y cuando tenga la palabra County 
data['ciudad'] = direccion_split.apply(
    lambda row: next((x.strip() for x in row if 'County' in str(x)), 'No definido'), axis=1
)

# Extraemos el estado  "Florida"
data['estado'] = direccion_split.apply(lambda row: 'Florida', axis=1)

# Extraemos  y validamos los códigos postales 
data['codigo_postal'] = direccion_split.apply(
    lambda row: next((re.search(r'\b\d{5}\b', str(x)).group(0) for x in row if re.search(r'\b\d{5}\b', str(x))), 'No definido'), axis=1
)

# Separamos el peis "United States"
data['pais'] = direccion_split.apply(lambda row: 'United States', axis=1)

# Reordenamos las columnas 
data = data[['nombre', 'direccion', 'ciudad', 'estado', 'pais', 'codigo_postal', 'longitud', 'latitud']]

# Eliminamos  duplicados
data.drop_duplicates(inplace=True)

# Guardamos el resultado
processed_file_path = 'H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2017.csv'
data.to_csv(processed_file_path, index=False)

# Análisamos de datos vacíos y duplicados
vacios = data.isnull().sum()
duplicados = data.duplicated().sum()

print("Análisis de datos:")
print("Valores vacíos por columna:")
print(vacios)
print(f"Número de filas duplicadas: {duplicados}")
print(data.head())

Análisis de datos:
Valores vacíos por columna:
nombre           0
direccion        0
ciudad           0
estado           0
pais             0
codigo_postal    0
longitud         0
latitud          0
dtype: int64
Número de filas duplicadas: 0
              nombre                               direccion  \
0   Allentown School               6180, Central School Road   
1    Alliance School               Alliance Road, Sink Creek   
2     Allison School          Lanier Grade, Jefferson County   
3  Alta Vista School           South Euclid Avenue, Sarasota   
4    Anderson School  Northeast 17th Avenue, Bradford County   

              ciudad   estado           pais codigo_postal   longitud  \
0  Santa Rosa County  Florida  United States         32570 -87.075245   
1     Jackson County  Florida  United States         32448 -85.136313   
2   Jefferson County  Florida  United States   No definido -83.826548   
3    Sarasota County  Florida  United States         34239 -82.518152   
4    Bra

Eliminamos la palabra county

In [19]:
# Eliminamos la palabra 'County' de las columnas 'direccion' y 'ciudad'
data['direccion'] = data['direccion'].str.replace(r'\bCounty\b', '', regex=True).str.strip()
data['ciudad'] = data['ciudad'].str.replace(r'\bCounty\b', '', regex=True).str.strip()
print(data[['direccion', 'ciudad']].head())

# Guardamos el resultado 
processed_file_path = 'H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2017.csv'
data.to_csv(processed_file_path, index=False)

                         direccion      ciudad
0        6180, Central School Road  Santa Rosa
1        Alliance Road, Sink Creek     Jackson
2          Lanier Grade, Jefferson   Jefferson
3    South Euclid Avenue, Sarasota    Sarasota
4  Northeast 17th Avenue, Bradford    Bradford


In [20]:
# Cargamos las rutas de los archivos
input_path = r"H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2017.csv"
output_path = r"H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2017.csv"
nombres_dim_path = "H:/Nueva carpeta/overpass-turbo.eu/nombres_unidades_educativas_dim.csv"
ciudades_dim_path = "H:/Nueva carpeta/overpass-turbo.eu/ciudades_dim.csv"

# Cargamos los catálogos
def load_catalog(path, key, value):
    catalog = {}
    if os.path.exists(path):
        with open(path, mode='r', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for row in reader:
                catalog[row[key]] = int(row[value])
    return catalog

# Guardamos los catálogos de ciudad y nombres
def save_catalog(path, catalog, key, value):
    with open(path, mode='w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=[value, key])
        writer.writeheader()
        for k, v in catalog.items():
            writer.writerow({value: v, key: k})

# Generamos IDs únicos
def get_id(value, catalog):
    if value not in catalog:
        catalog[value] = len(catalog) + 1
    return catalog[value]

# Cargamos catálogos existentes
nombres_dict = load_catalog(nombres_dim_path, "nombre", "id_nombre")
ciudades_dict = load_catalog(ciudades_dim_path, "ciudad", "id_ciudad")

# Procesamos archivo de entrada
def process_file(input_path, output_path):
    if not os.path.exists(input_path):
        raise FileNotFoundError(f"El archivo de entrada no existe: {input_path}")

    with open(input_path, mode='r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        original_fieldnames = reader.fieldnames

        # Validamos las columnas columnas requeridas
        required_cols = {"nombre", "direccion", "latitud", "longitud", "ciudad", "estado", "pais", "codigo_postal"}
        missing_cols = required_cols - set(original_fieldnames)
        if missing_cols:
            raise ValueError(f"El CSV no contiene las siguientes columnas requeridas: {', '.join(missing_cols)}")

        # Nombres de las columnas
        fieldnames = [
            "id_nombre", "nombre", "direccion", "latitud", "longitud",
            "id_ciudad", "ciudad", "estado", "pais", "codigo_postal"
        ]

        # ordenamos y creamos el archivo de salida
        with open(output_path, mode='w', newline='', encoding='utf-8') as outfile:
            writer = csv.DictWriter(outfile, fieldnames=fieldnames)
            writer.writeheader()

            for row in reader:
                id_nombre = get_id(row["nombre"], nombres_dict)
                id_ciudad = get_id(row["ciudad"], ciudades_dict)

                new_row = {
                    "id_nombre": id_nombre,
                    "nombre": row["nombre"],
                    "direccion": row["direccion"],
                    "latitud": row["latitud"],
                    "longitud": row["longitud"],
                    "id_ciudad": id_ciudad,
                    "ciudad": row["ciudad"],
                    "estado": row["estado"],
                    "pais": row["pais"],
                    "codigo_postal": row["codigo_postal"]
                }
                writer.writerow(new_row)

process_file(input_path, output_path)

# Guardamos el catálogos actualizados 
save_catalog(nombres_dim_path, nombres_dict, "nombre", "id_nombre")
save_catalog(ciudades_dim_path, ciudades_dict, "ciudad", "id_ciudad")

print("Proceso completado con éxito.")

Proceso completado con éxito.


Pasamos el archivo a formato parquet

In [21]:
# Cargamos el archivo CSV 
csv_path = r"H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2017.csv"

# Ruta del archivo Parquet de salida
parquet_path = r"H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2017.parquet"

df = pd.read_csv(csv_path)

# Guardamos el archivo Parquet
df.to_parquet(parquet_path, engine='pyarrow', index=False)

print(f"Archivo convertido exitosamente a Parquet: {parquet_path}")

Archivo convertido exitosamente a Parquet: H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2017.parquet


unidad educativa 2018

Extraemos la informacion de la columna id

In [22]:
# Cargamos  los datos 
archivo = "H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2018.csv"  
data = pd.read_csv(archivo)

# Renombramos  las columnas
data.rename(columns={
    'addr:street': 'direccion',
    'amenity': 'tipo',
    '@lat': 'latitud',
    '@lon': 'longitud'
}, inplace=True)

# Dividimos la columna direccion para extraer los datos
direccion_split = data['direccion'].fillna('').str.split(',', expand=True)

# Creamos las nuebas colunas a usar
data['nombre'] = direccion_split[0].str.strip()

# Extraemos la direccion combinando todas las partes de la dirección hasta el estado
data['direccion'] = direccion_split.apply(
    lambda row: ', '.join(filter(None, [row[1].strip(), row[2].strip()])), axis=1
)

# Extraemos la ciudad siempre y cuando tenga la palabra County 
data['ciudad'] = direccion_split.apply(
    lambda row: next((x.strip() for x in row if 'County' in str(x)), 'No definido'), axis=1
)

# Extraemos el estado  "Florida"
data['estado'] = direccion_split.apply(lambda row: 'Florida', axis=1)

# Extraemos  y validamos los códigos postales 
data['codigo_postal'] = direccion_split.apply(
    lambda row: next((re.search(r'\b\d{5}\b', str(x)).group(0) for x in row if re.search(r'\b\d{5}\b', str(x))), 'No definido'), axis=1
)

# Separamos el peis "United States"
data['pais'] = direccion_split.apply(lambda row: 'United States', axis=1)

# Reordenamos las columnas 
data = data[['nombre', 'direccion', 'ciudad', 'estado', 'pais', 'codigo_postal', 'longitud', 'latitud']]

# Eliminamos  duplicados
data.drop_duplicates(inplace=True)

# Guardamos el resultado
processed_file_path = 'H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2018.csv'
data.to_csv(processed_file_path, index=False)

# Análisamos de datos vacíos y duplicados
vacios = data.isnull().sum()
duplicados = data.duplicated().sum()

print("Análisis de datos:")
print("Valores vacíos por columna:")
print(vacios)
print(f"Número de filas duplicadas: {duplicados}")
print(data.head())

Análisis de datos:
Valores vacíos por columna:
nombre           0
direccion        0
ciudad           0
estado           0
pais             0
codigo_postal    0
longitud         0
latitud          0
dtype: int64
Número de filas duplicadas: 0
              nombre                               direccion  \
0   Allentown School               6180, Central School Road   
1    Alliance School               Alliance Road, Sink Creek   
2     Allison School          Lanier Grade, Jefferson County   
3  Alta Vista School           South Euclid Avenue, Sarasota   
4    Anderson School  Northeast 17th Avenue, Bradford County   

              ciudad   estado           pais codigo_postal   longitud  \
0  Santa Rosa County  Florida  United States         32570 -87.075245   
1     Jackson County  Florida  United States         32448 -85.136313   
2   Jefferson County  Florida  United States   No definido -83.826548   
3    Sarasota County  Florida  United States         34239 -82.518152   
4    Bra

Eliminamos la palabra county

In [23]:
# Eliminamos la palabra 'County' de las columnas 'direccion' y 'ciudad'
data['direccion'] = data['direccion'].str.replace(r'\bCounty\b', '', regex=True).str.strip()
data['ciudad'] = data['ciudad'].str.replace(r'\bCounty\b', '', regex=True).str.strip()
print(data[['direccion', 'ciudad']].head())

# Guardamos el resultado 
processed_file_path = 'H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2017.csv'
data.to_csv(processed_file_path, index=False)

                         direccion      ciudad
0        6180, Central School Road  Santa Rosa
1        Alliance Road, Sink Creek     Jackson
2          Lanier Grade, Jefferson   Jefferson
3    South Euclid Avenue, Sarasota    Sarasota
4  Northeast 17th Avenue, Bradford    Bradford


Creamos la columna id nombre e id ciudad

In [25]:
# Cargamos las rutas de los archivos
input_path = r"H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2018.csv"
output_path = r"H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2018.csv"
nombres_dim_path = "H:/Nueva carpeta/overpass-turbo.eu/nombres_unidades_educativas_dim.csv"
ciudades_dim_path = "H:/Nueva carpeta/overpass-turbo.eu/ciudades_dim.csv"

# Cargamos los catálogos
def load_catalog(path, key, value):
    catalog = {}
    if os.path.exists(path):
        with open(path, mode='r', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for row in reader:
                catalog[row[key]] = int(row[value])
    return catalog

# Guardamos los catálogos de ciudad y nombres
def save_catalog(path, catalog, key, value):
    with open(path, mode='w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=[value, key])
        writer.writeheader()
        for k, v in catalog.items():
            writer.writerow({value: v, key: k})

# Generamos IDs únicos
def get_id(value, catalog):
    if value not in catalog:
        catalog[value] = len(catalog) + 1
    return catalog[value]

# Cargamos catálogos existentes
nombres_dict = load_catalog(nombres_dim_path, "nombre", "id_nombre")
ciudades_dict = load_catalog(ciudades_dim_path, "ciudad", "id_ciudad")

# Procesamos archivo de entrada
def process_file(input_path, output_path):
    if not os.path.exists(input_path):
        raise FileNotFoundError(f"El archivo de entrada no existe: {input_path}")

    with open(input_path, mode='r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        original_fieldnames = reader.fieldnames

        # Validamos las columnas columnas requeridas
        required_cols = {"nombre", "direccion", "latitud", "longitud", "ciudad", "estado", "pais", "codigo_postal"}
        missing_cols = required_cols - set(original_fieldnames)
        if missing_cols:
            raise ValueError(f"El CSV no contiene las siguientes columnas requeridas: {', '.join(missing_cols)}")

        # Nombres de las columnas
        fieldnames = [
            "id_nombre", "nombre", "direccion", "latitud", "longitud",
            "id_ciudad", "ciudad", "estado", "pais", "codigo_postal"
        ]

        # ordenamos y creamos el archivo de salida
        with open(output_path, mode='w', newline='', encoding='utf-8') as outfile:
            writer = csv.DictWriter(outfile, fieldnames=fieldnames)
            writer.writeheader()

            for row in reader:
                id_nombre = get_id(row["nombre"], nombres_dict)
                id_ciudad = get_id(row["ciudad"], ciudades_dict)

                new_row = {
                    "id_nombre": id_nombre,
                    "nombre": row["nombre"],
                    "direccion": row["direccion"],
                    "latitud": row["latitud"],
                    "longitud": row["longitud"],
                    "id_ciudad": id_ciudad,
                    "ciudad": row["ciudad"],
                    "estado": row["estado"],
                    "pais": row["pais"],
                    "codigo_postal": row["codigo_postal"]
                }
                writer.writerow(new_row)

process_file(input_path, output_path)

# Guardamos el catálogos actualizados 
save_catalog(nombres_dim_path, nombres_dict, "nombre", "id_nombre")
save_catalog(ciudades_dim_path, ciudades_dict, "ciudad", "id_ciudad")

print("Proceso completado con éxito.")

Proceso completado con éxito.


Pasamos el archivo a formato .parquet 

In [27]:
# Cargamos el archivo CSV 
csv_path = r"H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2018.csv"

# Ruta del archivo Parquet de salida
parquet_path = r"H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2018.parquet"

df = pd.read_csv(csv_path)

# Guardamos el archivo Parquet
df.to_parquet(parquet_path, engine='pyarrow', index=False)

print(f"Archivo convertido exitosamente a Parquet: {parquet_path}")

Archivo convertido exitosamente a Parquet: H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2018.parquet


unidad educativa 2019

Extraemos la imformacion de la coluna direccion

In [28]:
# Cargamos  los datos 
archivo = "H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2019.csv"  
data = pd.read_csv(archivo)

# Renombramos  las columnas
data.rename(columns={
    'addr:street': 'direccion',
    'amenity': 'tipo',
    '@lat': 'latitud',
    '@lon': 'longitud'
}, inplace=True)

# Dividimos la columna direccion para extraer los datos
direccion_split = data['direccion'].fillna('').str.split(',', expand=True)

# Creamos las nuebas colunas a usar
data['nombre'] = direccion_split[0].str.strip()

# Extraemos la direccion combinando todas las partes de la dirección hasta el estado
data['direccion'] = direccion_split.apply(
    lambda row: ', '.join(filter(None, [row[1].strip(), row[2].strip()])), axis=1
)

# Extraemos la ciudad siempre y cuando tenga la palabra County 
data['ciudad'] = direccion_split.apply(
    lambda row: next((x.strip() for x in row if 'County' in str(x)), 'No definido'), axis=1
)

# Extraemos el estado  "Florida"
data['estado'] = direccion_split.apply(lambda row: 'Florida', axis=1)

# Extraemos  y validamos los códigos postales 
data['codigo_postal'] = direccion_split.apply(
    lambda row: next((re.search(r'\b\d{5}\b', str(x)).group(0) for x in row if re.search(r'\b\d{5}\b', str(x))), 'No definido'), axis=1
)

# Separamos el peis "United States"
data['pais'] = direccion_split.apply(lambda row: 'United States', axis=1)

# Reordenamos las columnas 
data = data[['nombre', 'direccion', 'ciudad', 'estado', 'pais', 'codigo_postal', 'longitud', 'latitud']]

# Eliminamos  duplicados
data.drop_duplicates(inplace=True)

# Guardamos el resultado
processed_file_path = 'H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2019.csv'
data.to_csv(processed_file_path, index=False)

# Análisamos de datos vacíos y duplicados
vacios = data.isnull().sum()
duplicados = data.duplicated().sum()

print("Análisis de datos:")
print("Valores vacíos por columna:")
print(vacios)
print(f"Número de filas duplicadas: {duplicados}")
print(data.head())

Análisis de datos:
Valores vacíos por columna:
nombre           0
direccion        0
ciudad           0
estado           0
pais             0
codigo_postal    0
longitud         0
latitud          0
dtype: int64
Número de filas duplicadas: 0
              nombre                                          direccion  \
0   Allentown School                          6180, Central School Road   
1    Alliance School                          Alliance Road, Sink Creek   
2  Alta Vista School                      South Euclid Avenue, Sarasota   
3    Anderson School             Northeast 17th Avenue, Bradford County   
4    Anderson School  North Merritt Street, Warrington Village Apart...   

              ciudad   estado           pais codigo_postal   longitud  \
0  Santa Rosa County  Florida  United States         32570 -87.075245   
1     Jackson County  Florida  United States         32448 -85.136313   
2    Sarasota County  Florida  United States         34239 -82.518152   
3    Bradford C

Eliminamos la palabra county

In [29]:
# Eliminamos la palabra 'County' de las columnas 'direccion' y 'ciudad'
data['direccion'] = data['direccion'].str.replace(r'\bCounty\b', '', regex=True).str.strip()
data['ciudad'] = data['ciudad'].str.replace(r'\bCounty\b', '', regex=True).str.strip()
print(data[['direccion', 'ciudad']].head())

# Guardamos el resultado 
processed_file_path = 'H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2019.csv'
data.to_csv(processed_file_path, index=False)

                                           direccion      ciudad
0                          6180, Central School Road  Santa Rosa
1                          Alliance Road, Sink Creek     Jackson
2                      South Euclid Avenue, Sarasota    Sarasota
3                    Northeast 17th Avenue, Bradford    Bradford
4  North Merritt Street, Warrington Village Apart...    Escambia


Creamos las columnas id nombre e id ciudad

In [30]:
# Cargamos las rutas de los archivos
input_path = r"H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2019.csv"
output_path = r"H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2019.csv"
nombres_dim_path = "H:/Nueva carpeta/overpass-turbo.eu/nombres_unidades_educativas_dim.csv"
ciudades_dim_path = "H:/Nueva carpeta/overpass-turbo.eu/ciudades_dim.csv"

# Cargamos los catálogos
def load_catalog(path, key, value):
    catalog = {}
    if os.path.exists(path):
        with open(path, mode='r', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for row in reader:
                catalog[row[key]] = int(row[value])
    return catalog

# Guardamos los catálogos de ciudad y nombres
def save_catalog(path, catalog, key, value):
    with open(path, mode='w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=[value, key])
        writer.writeheader()
        for k, v in catalog.items():
            writer.writerow({value: v, key: k})

# Generamos IDs únicos
def get_id(value, catalog):
    if value not in catalog:
        catalog[value] = len(catalog) + 1
    return catalog[value]

# Cargamos catálogos existentes
nombres_dict = load_catalog(nombres_dim_path, "nombre", "id_nombre")
ciudades_dict = load_catalog(ciudades_dim_path, "ciudad", "id_ciudad")

# Procesamos archivo de entrada
def process_file(input_path, output_path):
    if not os.path.exists(input_path):
        raise FileNotFoundError(f"El archivo de entrada no existe: {input_path}")

    with open(input_path, mode='r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        original_fieldnames = reader.fieldnames

        # Validamos las columnas columnas requeridas
        required_cols = {"nombre", "direccion", "latitud", "longitud", "ciudad", "estado", "pais", "codigo_postal"}
        missing_cols = required_cols - set(original_fieldnames)
        if missing_cols:
            raise ValueError(f"El CSV no contiene las siguientes columnas requeridas: {', '.join(missing_cols)}")

        # Nombres de las columnas
        fieldnames = [
            "id_nombre", "nombre", "direccion", "latitud", "longitud",
            "id_ciudad", "ciudad", "estado", "pais", "codigo_postal"
        ]

        # ordenamos y creamos el archivo de salida
        with open(output_path, mode='w', newline='', encoding='utf-8') as outfile:
            writer = csv.DictWriter(outfile, fieldnames=fieldnames)
            writer.writeheader()

            for row in reader:
                id_nombre = get_id(row["nombre"], nombres_dict)
                id_ciudad = get_id(row["ciudad"], ciudades_dict)

                new_row = {
                    "id_nombre": id_nombre,
                    "nombre": row["nombre"],
                    "direccion": row["direccion"],
                    "latitud": row["latitud"],
                    "longitud": row["longitud"],
                    "id_ciudad": id_ciudad,
                    "ciudad": row["ciudad"],
                    "estado": row["estado"],
                    "pais": row["pais"],
                    "codigo_postal": row["codigo_postal"]
                }
                writer.writerow(new_row)

process_file(input_path, output_path)

# Guardamos el catálogos actualizados 
save_catalog(nombres_dim_path, nombres_dict, "nombre", "id_nombre")
save_catalog(ciudades_dim_path, ciudades_dict, "ciudad", "id_ciudad")

print("Proceso completado con éxito.")

Proceso completado con éxito.


Pasamos el archivo a formato parquet

In [31]:
# Cargamos el archivo CSV 
csv_path = r"H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2019.csv"

# Ruta del archivo Parquet de salida
parquet_path = r"H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2019.parquet"

df = pd.read_csv(csv_path)

# Guardamos el archivo Parquet
df.to_parquet(parquet_path, engine='pyarrow', index=False)

print(f"Archivo convertido exitosamente a Parquet: {parquet_path}")

Archivo convertido exitosamente a Parquet: H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2019.parquet


unidad educativa 2020

Extraemos la informacion de la columna direccion

In [32]:
# Cargamos  los datos 
archivo = "H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2020.csv"  
data = pd.read_csv(archivo)

# Renombramos  las columnas
data.rename(columns={
    'addr:street': 'direccion',
    'amenity': 'tipo',
    '@lat': 'latitud',
    '@lon': 'longitud'
}, inplace=True)

# Dividimos la columna direccion para extraer los datos
direccion_split = data['direccion'].fillna('').str.split(',', expand=True)

# Creamos las nuebas colunas a usar
data['nombre'] = direccion_split[0].str.strip()

# Extraemos la direccion combinando todas las partes de la dirección hasta el estado
data['direccion'] = direccion_split.apply(
    lambda row: ', '.join(filter(None, [row[1].strip(), row[2].strip()])), axis=1
)

# Extraemos la ciudad siempre y cuando tenga la palabra County 
data['ciudad'] = direccion_split.apply(
    lambda row: next((x.strip() for x in row if 'County' in str(x)), 'No definido'), axis=1
)

# Extraemos el estado  "Florida"
data['estado'] = direccion_split.apply(lambda row: 'Florida', axis=1)

# Extraemos  y validamos los códigos postales 
data['codigo_postal'] = direccion_split.apply(
    lambda row: next((re.search(r'\b\d{5}\b', str(x)).group(0) for x in row if re.search(r'\b\d{5}\b', str(x))), 'No definido'), axis=1
)

# Separamos el peis "United States"
data['pais'] = direccion_split.apply(lambda row: 'United States', axis=1)

# Reordenamos las columnas 
data = data[['nombre', 'direccion', 'ciudad', 'estado', 'pais', 'codigo_postal', 'longitud', 'latitud']]

# Eliminamos  duplicados
data.drop_duplicates(inplace=True)

# Guardamos el resultado
processed_file_path = 'H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2020.csv'
data.to_csv(processed_file_path, index=False)

# Análisamos de datos vacíos y duplicados
vacios = data.isnull().sum()
duplicados = data.duplicated().sum()

print("Análisis de datos:")
print("Valores vacíos por columna:")
print(vacios)
print(f"Número de filas duplicadas: {duplicados}")
print(data.head())

Análisis de datos:
Valores vacíos por columna:
nombre           0
direccion        0
ciudad           0
estado           0
pais             0
codigo_postal    0
longitud         0
latitud          0
dtype: int64
Número de filas duplicadas: 0
              nombre                                          direccion  \
0   Allentown School                          6180, Central School Road   
1    Alliance School                          Alliance Road, Sink Creek   
2  Alta Vista School                      South Euclid Avenue, Sarasota   
3    Anderson School             Northeast 17th Avenue, Bradford County   
4    Anderson School  North Merritt Street, Warrington Village Apart...   

              ciudad   estado           pais codigo_postal   longitud  \
0  Santa Rosa County  Florida  United States         32570 -87.075245   
1     Jackson County  Florida  United States         32448 -85.136313   
2    Sarasota County  Florida  United States         34239 -82.518152   
3    Bradford C

Eliminamos la palabra county 

In [33]:
# Eliminamos la palabra 'County' de las columnas 'direccion' y 'ciudad'
data['direccion'] = data['direccion'].str.replace(r'\bCounty\b', '', regex=True).str.strip()
data['ciudad'] = data['ciudad'].str.replace(r'\bCounty\b', '', regex=True).str.strip()
print(data[['direccion', 'ciudad']].head())

# Guardamos el resultado 
processed_file_path = 'H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2020.csv'
data.to_csv(processed_file_path, index=False)

                                           direccion      ciudad
0                          6180, Central School Road  Santa Rosa
1                          Alliance Road, Sink Creek     Jackson
2                      South Euclid Avenue, Sarasota    Sarasota
3                    Northeast 17th Avenue, Bradford    Bradford
4  North Merritt Street, Warrington Village Apart...    Escambia


Creamos la columna id nombre e id ciudad

In [34]:
# Cargamos las rutas de los archivos
input_path = r"H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2020.csv"
output_path = r"H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2020.csv"
nombres_dim_path = "H:/Nueva carpeta/overpass-turbo.eu/nombres_unidades_educativas_dim.csv"
ciudades_dim_path = "H:/Nueva carpeta/overpass-turbo.eu/ciudades_dim.csv"

# Cargamos los catálogos
def load_catalog(path, key, value):
    catalog = {}
    if os.path.exists(path):
        with open(path, mode='r', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for row in reader:
                catalog[row[key]] = int(row[value])
    return catalog

# Guardamos los catálogos de ciudad y nombres
def save_catalog(path, catalog, key, value):
    with open(path, mode='w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=[value, key])
        writer.writeheader()
        for k, v in catalog.items():
            writer.writerow({value: v, key: k})

# Generamos IDs únicos
def get_id(value, catalog):
    if value not in catalog:
        catalog[value] = len(catalog) + 1
    return catalog[value]

# Cargamos catálogos existentes
nombres_dict = load_catalog(nombres_dim_path, "nombre", "id_nombre")
ciudades_dict = load_catalog(ciudades_dim_path, "ciudad", "id_ciudad")

# Procesamos archivo de entrada
def process_file(input_path, output_path):
    if not os.path.exists(input_path):
        raise FileNotFoundError(f"El archivo de entrada no existe: {input_path}")

    with open(input_path, mode='r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        original_fieldnames = reader.fieldnames

        # Validamos las columnas columnas requeridas
        required_cols = {"nombre", "direccion", "latitud", "longitud", "ciudad", "estado", "pais", "codigo_postal"}
        missing_cols = required_cols - set(original_fieldnames)
        if missing_cols:
            raise ValueError(f"El CSV no contiene las siguientes columnas requeridas: {', '.join(missing_cols)}")

        # Nombres de las columnas
        fieldnames = [
            "id_nombre", "nombre", "direccion", "latitud", "longitud",
            "id_ciudad", "ciudad", "estado", "pais", "codigo_postal"
        ]

        # ordenamos y creamos el archivo de salida
        with open(output_path, mode='w', newline='', encoding='utf-8') as outfile:
            writer = csv.DictWriter(outfile, fieldnames=fieldnames)
            writer.writeheader()

            for row in reader:
                id_nombre = get_id(row["nombre"], nombres_dict)
                id_ciudad = get_id(row["ciudad"], ciudades_dict)

                new_row = {
                    "id_nombre": id_nombre,
                    "nombre": row["nombre"],
                    "direccion": row["direccion"],
                    "latitud": row["latitud"],
                    "longitud": row["longitud"],
                    "id_ciudad": id_ciudad,
                    "ciudad": row["ciudad"],
                    "estado": row["estado"],
                    "pais": row["pais"],
                    "codigo_postal": row["codigo_postal"]
                }
                writer.writerow(new_row)

process_file(input_path, output_path)

# Guardamos el catálogos actualizados 
save_catalog(nombres_dim_path, nombres_dict, "nombre", "id_nombre")
save_catalog(ciudades_dim_path, ciudades_dict, "ciudad", "id_ciudad")

print("Proceso completado con éxito.")

Proceso completado con éxito.


Pasamos a punto parquet

In [35]:
# Cargamos el archivo CSV 
csv_path = r"H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2020.csv"

# Ruta del archivo Parquet de salida
parquet_path = r"H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2020.parquet"

df = pd.read_csv(csv_path)

# Guardamos el archivo Parquet
df.to_parquet(parquet_path, engine='pyarrow', index=False)

print(f"Archivo convertido exitosamente a Parquet: {parquet_path}")

Archivo convertido exitosamente a Parquet: H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2020.parquet


unidad educativa 2021

Extraemos la informacion de la columna direccion 

In [36]:
# Cargamos  los datos 
archivo = "H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2021.csv"  
data = pd.read_csv(archivo)

# Renombramos  las columnas
data.rename(columns={
    'addr:street': 'direccion',
    'amenity': 'tipo',
    '@lat': 'latitud',
    '@lon': 'longitud'
}, inplace=True)

# Dividimos la columna direccion para extraer los datos
direccion_split = data['direccion'].fillna('').str.split(',', expand=True)

# Creamos las nuebas colunas a usar
data['nombre'] = direccion_split[0].str.strip()

# Extraemos la direccion combinando todas las partes de la dirección hasta el estado
data['direccion'] = direccion_split.apply(
    lambda row: ', '.join(filter(None, [row[1].strip(), row[2].strip()])), axis=1
)

# Extraemos la ciudad siempre y cuando tenga la palabra County 
data['ciudad'] = direccion_split.apply(
    lambda row: next((x.strip() for x in row if 'County' in str(x)), 'No definido'), axis=1
)

# Extraemos el estado  "Florida"
data['estado'] = direccion_split.apply(lambda row: 'Florida', axis=1)

# Extraemos  y validamos los códigos postales 
data['codigo_postal'] = direccion_split.apply(
    lambda row: next((re.search(r'\b\d{5}\b', str(x)).group(0) for x in row if re.search(r'\b\d{5}\b', str(x))), 'No definido'), axis=1
)

# Separamos el peis "United States"
data['pais'] = direccion_split.apply(lambda row: 'United States', axis=1)

# Reordenamos las columnas 
data = data[['nombre', 'direccion', 'ciudad', 'estado', 'pais', 'codigo_postal', 'longitud', 'latitud']]

# Eliminamos  duplicados
data.drop_duplicates(inplace=True)

# Guardamos el resultado
processed_file_path = 'H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2021.csv'
data.to_csv(processed_file_path, index=False)

# Análisamos de datos vacíos y duplicados
vacios = data.isnull().sum()
duplicados = data.duplicated().sum()

print("Análisis de datos:")
print("Valores vacíos por columna:")
print(vacios)
print(f"Número de filas duplicadas: {duplicados}")
print(data.head())

Análisis de datos:
Valores vacíos por columna:
nombre           0
direccion        0
ciudad           0
estado           0
pais             0
codigo_postal    0
longitud         0
latitud          0
dtype: int64
Número de filas duplicadas: 0
              nombre                                          direccion  \
0   Allentown School                          6180, Central School Road   
1    Alliance School                          Alliance Road, Sink Creek   
2  Alta Vista School                      South Euclid Avenue, Sarasota   
3    Anderson School             Northeast 17th Avenue, Bradford County   
4    Anderson School  North Merritt Street, Warrington Village Apart...   

              ciudad   estado           pais codigo_postal   longitud  \
0  Santa Rosa County  Florida  United States         32570 -87.075245   
1     Jackson County  Florida  United States         32448 -85.136313   
2    Sarasota County  Florida  United States         34239 -82.518152   
3    Bradford C

Eliminamos la palabra county

In [37]:
# Eliminamos la palabra 'County' de las columnas 'direccion' y 'ciudad'
data['direccion'] = data['direccion'].str.replace(r'\bCounty\b', '', regex=True).str.strip()
data['ciudad'] = data['ciudad'].str.replace(r'\bCounty\b', '', regex=True).str.strip()
print(data[['direccion', 'ciudad']].head())

# Guardamos el resultado 
processed_file_path = 'H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2021.csv'
data.to_csv(processed_file_path, index=False)

                                           direccion      ciudad
0                          6180, Central School Road  Santa Rosa
1                          Alliance Road, Sink Creek     Jackson
2                      South Euclid Avenue, Sarasota    Sarasota
3                    Northeast 17th Avenue, Bradford    Bradford
4  North Merritt Street, Warrington Village Apart...    Escambia


Creamos las columnas id nombre e id ciudad

In [38]:
# Cargamos las rutas de los archivos
input_path = r"H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2021.csv"
output_path = r"H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2021.csv"
nombres_dim_path = "H:/Nueva carpeta/overpass-turbo.eu/nombres_unidades_educativas_dim.csv"
ciudades_dim_path = "H:/Nueva carpeta/overpass-turbo.eu/ciudades_dim.csv"

# Cargamos los catálogos
def load_catalog(path, key, value):
    catalog = {}
    if os.path.exists(path):
        with open(path, mode='r', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for row in reader:
                catalog[row[key]] = int(row[value])
    return catalog

# Guardamos los catálogos de ciudad y nombres
def save_catalog(path, catalog, key, value):
    with open(path, mode='w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=[value, key])
        writer.writeheader()
        for k, v in catalog.items():
            writer.writerow({value: v, key: k})

# Generamos IDs únicos
def get_id(value, catalog):
    if value not in catalog:
        catalog[value] = len(catalog) + 1
    return catalog[value]

# Cargamos catálogos existentes
nombres_dict = load_catalog(nombres_dim_path, "nombre", "id_nombre")
ciudades_dict = load_catalog(ciudades_dim_path, "ciudad", "id_ciudad")

# Procesamos archivo de entrada
def process_file(input_path, output_path):
    if not os.path.exists(input_path):
        raise FileNotFoundError(f"El archivo de entrada no existe: {input_path}")

    with open(input_path, mode='r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        original_fieldnames = reader.fieldnames

        # Validamos las columnas columnas requeridas
        required_cols = {"nombre", "direccion", "latitud", "longitud", "ciudad", "estado", "pais", "codigo_postal"}
        missing_cols = required_cols - set(original_fieldnames)
        if missing_cols:
            raise ValueError(f"El CSV no contiene las siguientes columnas requeridas: {', '.join(missing_cols)}")

        # Nombres de las columnas
        fieldnames = [
            "id_nombre", "nombre", "direccion", "latitud", "longitud",
            "id_ciudad", "ciudad", "estado", "pais", "codigo_postal"
        ]

        # ordenamos y creamos el archivo de salida
        with open(output_path, mode='w', newline='', encoding='utf-8') as outfile:
            writer = csv.DictWriter(outfile, fieldnames=fieldnames)
            writer.writeheader()

            for row in reader:
                id_nombre = get_id(row["nombre"], nombres_dict)
                id_ciudad = get_id(row["ciudad"], ciudades_dict)

                new_row = {
                    "id_nombre": id_nombre,
                    "nombre": row["nombre"],
                    "direccion": row["direccion"],
                    "latitud": row["latitud"],
                    "longitud": row["longitud"],
                    "id_ciudad": id_ciudad,
                    "ciudad": row["ciudad"],
                    "estado": row["estado"],
                    "pais": row["pais"],
                    "codigo_postal": row["codigo_postal"]
                }
                writer.writerow(new_row)

process_file(input_path, output_path)

# Guardamos el catálogos actualizados 
save_catalog(nombres_dim_path, nombres_dict, "nombre", "id_nombre")
save_catalog(ciudades_dim_path, ciudades_dict, "ciudad", "id_ciudad")

print("Proceso completado con éxito.")

Proceso completado con éxito.


pasamos el archivo a formato .parquet

In [39]:
# Cargamos el archivo CSV 
csv_path = r"H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2021.csv"

# Ruta del archivo Parquet de salida
parquet_path = r"H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2021.parquet"

df = pd.read_csv(csv_path)

# Guardamos el archivo Parquet
df.to_parquet(parquet_path, engine='pyarrow', index=False)

print(f"Archivo convertido exitosamente a Parquet: {parquet_path}")

Archivo convertido exitosamente a Parquet: H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2021.parquet


unidad educativa 2022

Extraemos la informacion de la columna direccion

In [40]:
# Cargamos  los datos 
archivo = "H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2022.csv"  
data = pd.read_csv(archivo)

# Renombramos  las columnas
data.rename(columns={
    'addr:street': 'direccion',
    'amenity': 'tipo',
    '@lat': 'latitud',
    '@lon': 'longitud'
}, inplace=True)

# Dividimos la columna direccion para extraer los datos
direccion_split = data['direccion'].fillna('').str.split(',', expand=True)

# Creamos las nuebas colunas a usar
data['nombre'] = direccion_split[0].str.strip()

# Extraemos la direccion combinando todas las partes de la dirección hasta el estado
data['direccion'] = direccion_split.apply(
    lambda row: ', '.join(filter(None, [row[1].strip(), row[2].strip()])), axis=1
)

# Extraemos la ciudad siempre y cuando tenga la palabra County 
data['ciudad'] = direccion_split.apply(
    lambda row: next((x.strip() for x in row if 'County' in str(x)), 'No definido'), axis=1
)

# Extraemos el estado  "Florida"
data['estado'] = direccion_split.apply(lambda row: 'Florida', axis=1)

# Extraemos  y validamos los códigos postales 
data['codigo_postal'] = direccion_split.apply(
    lambda row: next((re.search(r'\b\d{5}\b', str(x)).group(0) for x in row if re.search(r'\b\d{5}\b', str(x))), 'No definido'), axis=1
)

# Separamos el peis "United States"
data['pais'] = direccion_split.apply(lambda row: 'United States', axis=1)

# Reordenamos las columnas 
data = data[['nombre', 'direccion', 'ciudad', 'estado', 'pais', 'codigo_postal', 'longitud', 'latitud']]

# Eliminamos  duplicados
data.drop_duplicates(inplace=True)

# Guardamos el resultado
processed_file_path = 'H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2022.csv'
data.to_csv(processed_file_path, index=False)

# Análisamos de datos vacíos y duplicados
vacios = data.isnull().sum()
duplicados = data.duplicated().sum()

print("Análisis de datos:")
print("Valores vacíos por columna:")
print(vacios)
print(f"Número de filas duplicadas: {duplicados}")
print(data.head())

Análisis de datos:
Valores vacíos por columna:
nombre           0
direccion        0
ciudad           0
estado           0
pais             0
codigo_postal    0
longitud         0
latitud          0
dtype: int64
Número de filas duplicadas: 0
              nombre                                          direccion  \
0   Allentown School                          6180, Central School Road   
1    Alliance School                          Alliance Road, Sink Creek   
2  Alta Vista School                      South Euclid Avenue, Sarasota   
3    Anderson School             Northeast 17th Avenue, Bradford County   
4    Anderson School  North Merritt Street, Warrington Village Apart...   

              ciudad   estado           pais codigo_postal   longitud  \
0  Santa Rosa County  Florida  United States         32570 -87.075245   
1     Jackson County  Florida  United States         32448 -85.136313   
2    Sarasota County  Florida  United States         34239 -82.518152   
3    Bradford C

Eliminamos la palabra county

In [ ]:
# Eliminamos la palabra 'County' de las columnas 'direccion' y 'ciudad'
data['direccion'] = data['direccion'].str.replace(r'\bCounty\b', '', regex=True).str.strip()
data['ciudad'] = data['ciudad'].str.replace(r'\bCounty\b', '', regex=True).str.strip()
print(data[['direccion', 'ciudad']].head())

# Guardamos el resultado 
processed_file_path = 'H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2014.csv'
data.to_csv(processed_file_path, index=False)

Creamos las columnas id nombre e id ciudad

In [42]:
# Cargamos las rutas de los archivos
input_path = r"H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2022.csv"
output_path = r"H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2022.csv"
nombres_dim_path = "H:/Nueva carpeta/overpass-turbo.eu/nombres_unidades_educativas_dim.csv"
ciudades_dim_path = "H:/Nueva carpeta/overpass-turbo.eu/ciudades_dim.csv"

# Cargamos los catálogos
def load_catalog(path, key, value):
    catalog = {}
    if os.path.exists(path):
        with open(path, mode='r', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for row in reader:
                catalog[row[key]] = int(row[value])
    return catalog

# Guardamos los catálogos de ciudad y nombres
def save_catalog(path, catalog, key, value):
    with open(path, mode='w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=[value, key])
        writer.writeheader()
        for k, v in catalog.items():
            writer.writerow({value: v, key: k})

# Generamos IDs únicos
def get_id(value, catalog):
    if value not in catalog:
        catalog[value] = len(catalog) + 1
    return catalog[value]

# Cargamos catálogos existentes
nombres_dict = load_catalog(nombres_dim_path, "nombre", "id_nombre")
ciudades_dict = load_catalog(ciudades_dim_path, "ciudad", "id_ciudad")

# Procesamos archivo de entrada
def process_file(input_path, output_path):
    if not os.path.exists(input_path):
        raise FileNotFoundError(f"El archivo de entrada no existe: {input_path}")

    with open(input_path, mode='r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        original_fieldnames = reader.fieldnames

        # Validamos las columnas columnas requeridas
        required_cols = {"nombre", "direccion", "latitud", "longitud", "ciudad", "estado", "pais", "codigo_postal"}
        missing_cols = required_cols - set(original_fieldnames)
        if missing_cols:
            raise ValueError(f"El CSV no contiene las siguientes columnas requeridas: {', '.join(missing_cols)}")

        # Nombres de las columnas
        fieldnames = [
            "id_nombre", "nombre", "direccion", "latitud", "longitud",
            "id_ciudad", "ciudad", "estado", "pais", "codigo_postal"
        ]

        # ordenamos y creamos el archivo de salida
        with open(output_path, mode='w', newline='', encoding='utf-8') as outfile:
            writer = csv.DictWriter(outfile, fieldnames=fieldnames)
            writer.writeheader()

            for row in reader:
                id_nombre = get_id(row["nombre"], nombres_dict)
                id_ciudad = get_id(row["ciudad"], ciudades_dict)

                new_row = {
                    "id_nombre": id_nombre,
                    "nombre": row["nombre"],
                    "direccion": row["direccion"],
                    "latitud": row["latitud"],
                    "longitud": row["longitud"],
                    "id_ciudad": id_ciudad,
                    "ciudad": row["ciudad"],
                    "estado": row["estado"],
                    "pais": row["pais"],
                    "codigo_postal": row["codigo_postal"]
                }
                writer.writerow(new_row)

process_file(input_path, output_path)

# Guardamos el catálogos actualizados 
save_catalog(nombres_dim_path, nombres_dict, "nombre", "id_nombre")
save_catalog(ciudades_dim_path, ciudades_dict, "ciudad", "id_ciudad")

print("Proceso completado con éxito.")

Proceso completado con éxito.


Pasamos el archivo a formato parquet

In [44]:
# Cargamos el archivo CSV 
csv_path = r"H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2022.csv"

# Ruta del archivo Parquet de salida
parquet_path = r"H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2022.parquet"

df = pd.read_csv(csv_path)

# Guardamos el archivo Parquet
df.to_parquet(parquet_path, engine='pyarrow', index=False)

print(f"Archivo convertido exitosamente a Parquet: {parquet_path}")

Archivo convertido exitosamente a Parquet: H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2022.parquet


unidad educativa 2023

Extraemos la informacion de la columna direccion

In [45]:
# Cargamos  los datos 
archivo = "H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2023.csv"  
data = pd.read_csv(archivo)

# Renombramos  las columnas
data.rename(columns={
    'addr:street': 'direccion',
    'amenity': 'tipo',
    '@lat': 'latitud',
    '@lon': 'longitud'
}, inplace=True)

# Dividimos la columna direccion para extraer los datos
direccion_split = data['direccion'].fillna('').str.split(',', expand=True)

# Creamos las nuebas colunas a usar
data['nombre'] = direccion_split[0].str.strip()

# Extraemos la direccion combinando todas las partes de la dirección hasta el estado
data['direccion'] = direccion_split.apply(
    lambda row: ', '.join(filter(None, [row[1].strip(), row[2].strip()])), axis=1
)

# Extraemos la ciudad siempre y cuando tenga la palabra County 
data['ciudad'] = direccion_split.apply(
    lambda row: next((x.strip() for x in row if 'County' in str(x)), 'No definido'), axis=1
)

# Extraemos el estado  "Florida"
data['estado'] = direccion_split.apply(lambda row: 'Florida', axis=1)

# Extraemos  y validamos los códigos postales 
data['codigo_postal'] = direccion_split.apply(
    lambda row: next((re.search(r'\b\d{5}\b', str(x)).group(0) for x in row if re.search(r'\b\d{5}\b', str(x))), 'No definido'), axis=1
)

# Separamos el peis "United States"
data['pais'] = direccion_split.apply(lambda row: 'United States', axis=1)

# Reordenamos las columnas 
data = data[['nombre', 'direccion', 'ciudad', 'estado', 'pais', 'codigo_postal', 'longitud', 'latitud']]

# Eliminamos  duplicados
data.drop_duplicates(inplace=True)

# Guardamos el resultado
processed_file_path = 'H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2023.csv'
data.to_csv(processed_file_path, index=False)

# Análisamos de datos vacíos y duplicados
vacios = data.isnull().sum()
duplicados = data.duplicated().sum()

print("Análisis de datos:")
print("Valores vacíos por columna:")
print(vacios)
print(f"Número de filas duplicadas: {duplicados}")
print(data.head())

Análisis de datos:
Valores vacíos por columna:
nombre           0
direccion        0
ciudad           0
estado           0
pais             0
codigo_postal    0
longitud         0
latitud          0
dtype: int64
Número de filas duplicadas: 0
              nombre                                          direccion  \
0   Allentown School                          6180, Central School Road   
1    Alliance School                          Alliance Road, Sink Creek   
2  Alta Vista School                      South Euclid Avenue, Sarasota   
3    Anderson School             Northeast 17th Avenue, Bradford County   
4    Anderson School  North Merritt Street, Warrington Village Apart...   

              ciudad   estado           pais codigo_postal   longitud  \
0  Santa Rosa County  Florida  United States         32570 -87.075245   
1     Jackson County  Florida  United States         32448 -85.136313   
2    Sarasota County  Florida  United States         34239 -82.518152   
3    Bradford C

Eliminamos la palabra county

In [46]:
# Eliminamos la palabra 'County' de las columnas 'direccion' y 'ciudad'
data['direccion'] = data['direccion'].str.replace(r'\bCounty\b', '', regex=True).str.strip()
data['ciudad'] = data['ciudad'].str.replace(r'\bCounty\b', '', regex=True).str.strip()
print(data[['direccion', 'ciudad']].head())

# Guardamos el resultado 
processed_file_path = 'H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2023.csv'
data.to_csv(processed_file_path, index=False)

                                           direccion      ciudad
0                          6180, Central School Road  Santa Rosa
1                          Alliance Road, Sink Creek     Jackson
2                      South Euclid Avenue, Sarasota    Sarasota
3                    Northeast 17th Avenue, Bradford    Bradford
4  North Merritt Street, Warrington Village Apart...    Escambia


Creamos la columna id nombre e id ciudad

In [47]:
# Cargamos las rutas de los archivos
input_path = r"H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2023.csv"
output_path = r"H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2023.csv"
nombres_dim_path = "H:/Nueva carpeta/overpass-turbo.eu/nombres_unidades_educativas_dim.csv"
ciudades_dim_path = "H:/Nueva carpeta/overpass-turbo.eu/ciudades_dim.csv"

# Cargamos los catálogos
def load_catalog(path, key, value):
    catalog = {}
    if os.path.exists(path):
        with open(path, mode='r', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for row in reader:
                catalog[row[key]] = int(row[value])
    return catalog

# Guardamos los catálogos de ciudad y nombres
def save_catalog(path, catalog, key, value):
    with open(path, mode='w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=[value, key])
        writer.writeheader()
        for k, v in catalog.items():
            writer.writerow({value: v, key: k})

# Generamos IDs únicos
def get_id(value, catalog):
    if value not in catalog:
        catalog[value] = len(catalog) + 1
    return catalog[value]

# Cargamos catálogos existentes
nombres_dict = load_catalog(nombres_dim_path, "nombre", "id_nombre")
ciudades_dict = load_catalog(ciudades_dim_path, "ciudad", "id_ciudad")

# Procesamos archivo de entrada
def process_file(input_path, output_path):
    if not os.path.exists(input_path):
        raise FileNotFoundError(f"El archivo de entrada no existe: {input_path}")

    with open(input_path, mode='r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        original_fieldnames = reader.fieldnames

        # Validamos las columnas columnas requeridas
        required_cols = {"nombre", "direccion", "latitud", "longitud", "ciudad", "estado", "pais", "codigo_postal"}
        missing_cols = required_cols - set(original_fieldnames)
        if missing_cols:
            raise ValueError(f"El CSV no contiene las siguientes columnas requeridas: {', '.join(missing_cols)}")

        # Nombres de las columnas
        fieldnames = [
            "id_nombre", "nombre", "direccion", "latitud", "longitud",
            "id_ciudad", "ciudad", "estado", "pais", "codigo_postal"
        ]

        # ordenamos y creamos el archivo de salida
        with open(output_path, mode='w', newline='', encoding='utf-8') as outfile:
            writer = csv.DictWriter(outfile, fieldnames=fieldnames)
            writer.writeheader()

            for row in reader:
                id_nombre = get_id(row["nombre"], nombres_dict)
                id_ciudad = get_id(row["ciudad"], ciudades_dict)

                new_row = {
                    "id_nombre": id_nombre,
                    "nombre": row["nombre"],
                    "direccion": row["direccion"],
                    "latitud": row["latitud"],
                    "longitud": row["longitud"],
                    "id_ciudad": id_ciudad,
                    "ciudad": row["ciudad"],
                    "estado": row["estado"],
                    "pais": row["pais"],
                    "codigo_postal": row["codigo_postal"]
                }
                writer.writerow(new_row)

process_file(input_path, output_path)

# Guardamos el catálogos actualizados 
save_catalog(nombres_dim_path, nombres_dict, "nombre", "id_nombre")
save_catalog(ciudades_dim_path, ciudades_dict, "ciudad", "id_ciudad")

print("Proceso completado con éxito.")

Proceso completado con éxito.


Pasamos el archivo a formato parquet

In [48]:
# Cargamos el archivo CSV 
csv_path = r"H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2023.csv"

# Ruta del archivo Parquet de salida
parquet_path = r"H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2023.parquet"

df = pd.read_csv(csv_path)

# Guardamos el archivo Parquet
df.to_parquet(parquet_path, engine='pyarrow', index=False)

print(f"Archivo convertido exitosamente a Parquet: {parquet_path}")

Archivo convertido exitosamente a Parquet: H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2023.parquet


unidad educativa 2024

Extraemos la imformacion de la coluna direccion

In [49]:
# Cargamos  los datos 
archivo = "H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2024.csv"  
data = pd.read_csv(archivo)

# Renombramos  las columnas
data.rename(columns={
    'addr:street': 'direccion',
    'amenity': 'tipo',
    '@lat': 'latitud',
    '@lon': 'longitud'
}, inplace=True)

# Dividimos la columna direccion para extraer los datos
direccion_split = data['direccion'].fillna('').str.split(',', expand=True)

# Creamos las nuebas colunas a usar
data['nombre'] = direccion_split[0].str.strip()

# Extraemos la direccion combinando todas las partes de la dirección hasta el estado
data['direccion'] = direccion_split.apply(
    lambda row: ', '.join(filter(None, [row[1].strip(), row[2].strip()])), axis=1
)

# Extraemos la ciudad siempre y cuando tenga la palabra County 
data['ciudad'] = direccion_split.apply(
    lambda row: next((x.strip() for x in row if 'County' in str(x)), 'No definido'), axis=1
)

# Extraemos el estado  "Florida"
data['estado'] = direccion_split.apply(lambda row: 'Florida', axis=1)

# Extraemos  y validamos los códigos postales 
data['codigo_postal'] = direccion_split.apply(
    lambda row: next((re.search(r'\b\d{5}\b', str(x)).group(0) for x in row if re.search(r'\b\d{5}\b', str(x))), 'No definido'), axis=1
)

# Separamos el peis "United States"
data['pais'] = direccion_split.apply(lambda row: 'United States', axis=1)

# Reordenamos las columnas 
data = data[['nombre', 'direccion', 'ciudad', 'estado', 'pais', 'codigo_postal', 'longitud', 'latitud']]

# Eliminamos  duplicados
data.drop_duplicates(inplace=True)

# Guardamos el resultado
processed_file_path = 'H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2024.csv'
data.to_csv(processed_file_path, index=False)

# Análisamos de datos vacíos y duplicados
vacios = data.isnull().sum()
duplicados = data.duplicated().sum()

print("Análisis de datos:")
print("Valores vacíos por columna:")
print(vacios)
print(f"Número de filas duplicadas: {duplicados}")
print(data.head())

Análisis de datos:
Valores vacíos por columna:
nombre           0
direccion        0
ciudad           0
estado           0
pais             0
codigo_postal    0
longitud         0
latitud          0
dtype: int64
Número de filas duplicadas: 0
              nombre                                          direccion  \
0   Allentown School                          6180, Central School Road   
1    Alliance School                          Alliance Road, Sink Creek   
2  Alta Vista School                      South Euclid Avenue, Sarasota   
3    Anderson School             Northeast 17th Avenue, Bradford County   
4    Anderson School  North Merritt Street, Warrington Village Apart...   

              ciudad   estado           pais codigo_postal   longitud  \
0  Santa Rosa County  Florida  United States         32570 -87.075245   
1     Jackson County  Florida  United States         32448 -85.136313   
2    Sarasota County  Florida  United States         34239 -82.518152   
3    Bradford C

Eliminamos la palabra county

In [ ]:
# Eliminamos la palabra 'County' de las columnas 'direccion' y 'ciudad'
data['direccion'] = data['direccion'].str.replace(r'\bCounty\b', '', regex=True).str.strip()
data['ciudad'] = data['ciudad'].str.replace(r'\bCounty\b', '', regex=True).str.strip()
print(data[['direccion', 'ciudad']].head())

# Guardamos el resultado 
processed_file_path = 'H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2024.csv'
data.to_csv(processed_file_path, index=False)

Creamos la columna id nombre e id ciudad

In [50]:
# Cargamos las rutas de los archivos
input_path = r"H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2024.csv"
output_path = r"H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2024.csv"
nombres_dim_path = "H:/Nueva carpeta/overpass-turbo.eu/nombres_unidades_educativas_dim.csv"
ciudades_dim_path = "H:/Nueva carpeta/overpass-turbo.eu/ciudades_dim.csv"

# Cargamos los catálogos
def load_catalog(path, key, value):
    catalog = {}
    if os.path.exists(path):
        with open(path, mode='r', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for row in reader:
                catalog[row[key]] = int(row[value])
    return catalog

# Guardamos los catálogos de ciudad y nombres
def save_catalog(path, catalog, key, value):
    with open(path, mode='w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=[value, key])
        writer.writeheader()
        for k, v in catalog.items():
            writer.writerow({value: v, key: k})

# Generamos IDs únicos
def get_id(value, catalog):
    if value not in catalog:
        catalog[value] = len(catalog) + 1
    return catalog[value]

# Cargamos catálogos existentes
nombres_dict = load_catalog(nombres_dim_path, "nombre", "id_nombre")
ciudades_dict = load_catalog(ciudades_dim_path, "ciudad", "id_ciudad")

# Procesamos archivo de entrada
def process_file(input_path, output_path):
    if not os.path.exists(input_path):
        raise FileNotFoundError(f"El archivo de entrada no existe: {input_path}")

    with open(input_path, mode='r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        original_fieldnames = reader.fieldnames

        # Validamos las columnas columnas requeridas
        required_cols = {"nombre", "direccion", "latitud", "longitud", "ciudad", "estado", "pais", "codigo_postal"}
        missing_cols = required_cols - set(original_fieldnames)
        if missing_cols:
            raise ValueError(f"El CSV no contiene las siguientes columnas requeridas: {', '.join(missing_cols)}")

        # Nombres de las columnas
        fieldnames = [
            "id_nombre", "nombre", "direccion", "latitud", "longitud",
            "id_ciudad", "ciudad", "estado", "pais", "codigo_postal"
        ]

        # ordenamos y creamos el archivo de salida
        with open(output_path, mode='w', newline='', encoding='utf-8') as outfile:
            writer = csv.DictWriter(outfile, fieldnames=fieldnames)
            writer.writeheader()

            for row in reader:
                id_nombre = get_id(row["nombre"], nombres_dict)
                id_ciudad = get_id(row["ciudad"], ciudades_dict)

                new_row = {
                    "id_nombre": id_nombre,
                    "nombre": row["nombre"],
                    "direccion": row["direccion"],
                    "latitud": row["latitud"],
                    "longitud": row["longitud"],
                    "id_ciudad": id_ciudad,
                    "ciudad": row["ciudad"],
                    "estado": row["estado"],
                    "pais": row["pais"],
                    "codigo_postal": row["codigo_postal"]
                }
                writer.writerow(new_row)

process_file(input_path, output_path)

# Guardamos el catálogos actualizados 
save_catalog(nombres_dim_path, nombres_dict, "nombre", "id_nombre")
save_catalog(ciudades_dim_path, ciudades_dict, "ciudad", "id_ciudad")

print("Proceso completado con éxito.")

Proceso completado con éxito.


Pasamos el archivo a formato parquet

In [51]:
# Cargamos el archivo CSV 
csv_path = r"H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2024.csv"

# Ruta del archivo Parquet de salida
parquet_path = r"H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2024.parquet"

df = pd.read_csv(csv_path)

# Guardamos el archivo Parquet
df.to_parquet(parquet_path, engine='pyarrow', index=False)

print(f"Archivo convertido exitosamente a Parquet: {parquet_path}")

Archivo convertido exitosamente a Parquet: H:/Nueva carpeta/overpass-turbo.eu/unidad educativa/unidad educativa 2024.parquet
